In [1]:
!pip install -q transformers accelerate
!pip install -U bitsandbytes
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"  # Critical for T4

from PIL import Image
import torch
from transformers import (
    LlavaNextProcessor,
    LlavaNextForConditionalGeneration,
    BitsAndBytesConfig
)

In [20]:
import os
import gc
import torch
from PIL import Image
from transformers import (
    LlavaNextProcessor,
    LlavaNextForConditionalGeneration,
    BitsAndBytesConfig
)

# ====== MEMORY MANAGEMENT ======
def clear_memory():
    torch.cuda.empty_cache()
    gc.collect()

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# ====== QUANTIZATION ======
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# ====== MODEL LOADING ======
model_id = "llava-hf/llava-v1.6-vicuna-7b-hf"
processor = LlavaNextProcessor.from_pretrained(model_id)
model = LlavaNextForConditionalGeneration.from_pretrained(
    model_id,
    quantization_config=quant_config,
    device_map="auto",
    torch_dtype=torch.float16
)

# ====== IMAGE PROCESSING ======
image = Image.open("/content/drive/MyDrive/outputy.png").convert("RGB").resize((448, 448))

# ====== OPTIMIZED PROMPT ======
# ====== IMPROVED PROMPT STRUCTURE ======
prompt = "[INST] <image>\nCreate concise step-by-step DIY instructions with the materials used. [/INST]"

# ====== ENHANCED GENERATION CONFIG ======
generation_config = {
    "max_new_tokens": 400,
    "temperature": 0.1,  # More deterministic
    "top_p": 0.85,
    "repetition_penalty": 1.35,
    "num_beams": 2,
    "do_sample": False,  # Disable random sampling
    "pad_token_id": processor.tokenizer.eos_token_id,
}


# ====== EXECUTION ======
with torch.inference_mode():
    clear_memory()
    inputs = processor(prompt, image, return_tensors="pt").to("cuda", torch.float16)

    output = model.generate(**inputs, **generation_config)

    # Clean output formatting
    full_text = processor.decode(output[0], skip_special_tokens=True)
    print('\n'.join([line.strip() for line in full_text.split('\n') if line.strip()]))

# Cleanup
del inputs, output
clear_memory()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

[INST]
Create concise step-by-step DIY instructions with the materials used. [/INST]
Materials:
1. Aluminum can
2. Candle wax
3. Wick
4. Matches or a lighter
5. Wooden table or board
6. Paper plate or napkin
Instructions:
1. Clean the aluminum can thoroughly to remove any residue or debris.
2. Melt the candle wax in a double boiler or a heat-resistant bowl over low heat.
3. Dip the wick into the melted wax, ensuring it is fully submerged.
4. Remove the wick from the wax and allow it to drain for a few seconds.
5. Place the wick on the paper plate or napkin.
6. Light the wick using a match or a lighter.
7. Once the wick is lit, carefully place the lit wick onto the melted wax in the aluminum can.
8. Allow the wax to cool and harden completely.
9. Once the wax is hardened, remove the wick from the can and trim the wick to the desired length.
10. Place the lit candle in the aluminum can on the wooden table or board.
11. Enjoy your homemade candle.
Please note that this is a simple DIY pro